<a href="https://colab.research.google.com/github/perrin-isir/xomx-tutorials/blob/main/tutorials/xomx_hla.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>
<a id="raw-url" href="https://raw.githubusercontent.com/perrin-isir/xomx-tutorials/main/tutorials/xomx_hla.ipynb" download> <img align="left" src="https://img.shields.io/badge/Github-Download%20(Right%20click%20%2B%20Save%20link%20as...)-blue" alt="Download (Right click + Save link as)" title="Download Notebook"></a>

# *xomx tutorial:* **tissue prediction based on HLA-presented peptides**

In [ ]:
# imports:
import os
import joblib
from IPython.display import clear_output
try:
    import xomx
except ImportError:
    !pip install xomx
    clear_output()
    import xomx
try:
    import scanpy as sc
except ImportError:
    !pip install scanpy
    clear_output()
    import scanpy as sc
try:
    import mhcflurry
except ImportError:
    !pip install mhcflurry
    !mhcflurry-downloads fetch models_class1_presentation
    clear_output()
    import mhcflurry
try:
    import trimap
except ImportError:
    !pip install trimap
    clear_output()
    import trimap
import numpy as np
import pandas as pd
import umap

In [ ]:
save_dir = os.path.join(os.path.expanduser("~"), "results", "xomx-tutorials", "xomx_hla")  # the default directory in which results are stored
os.makedirs(save_dir, exist_ok=True)

The HLA Ligand Atlas is a resource of natural HLA ligands presented on benign tissues.  
We first gather in a dict (`dfs`) 4 pandas dataframes from the HLA Ligand Atlas: 
- `dfs["peptides"]`: the list of peptide sequences with their id,
- `dfs["donors"]`: the list of donors and their alleles,
- `dfs["sample_hits"]`: for all the peptide sequences, the donors and tissues in which they have been found, and their HLA class,
- `dfs["aggregated"]`: one row per peptide sequence, with the HLA class of the peptide, and the list of donor alleles and tissues associated with the peptide. 

In [ ]:
base_url = "http://hla-ligand-atlas.org/rel/2020.12/"
filenames = ["peptides", "donors", "sample_hits", "aggregated"]
dfs = {}
for nm in filenames:
    if not os.path.isfile(os.path.join(save_dir, nm + ".joblib")):
        dfs[nm] = pd.read_csv(base_url + nm + ".tsv.gz", sep="\t")
        joblib.dump(dfs[nm], os.path.join(save_dir, nm + ".joblib"))
    else:
        dfs[nm] = joblib.load(os.path.join(save_dir, nm + ".joblib"))

We compute the set of all alleles present in the database:

In [ ]:
alleles_ = sorted(list(set(np.concatenate([allele.split(",") for allele in dfs["aggregated"].donor_alleles]))))

In this list, the alleles start with one of the 3 prefixes "n/", "w/" and "s/", which characterize binding predictions of peptides:  
- "n/": predicted non-binder donor allele
- "w/": predicted weak binder donor allele
- "s/": predicted strong binder donor allele

For example, the peptide with id 22 has been found in donors with the following alleles:

In [ ]:
list(dfs["aggregated"][dfs["aggregated"].peptide_sequence_id == 22].donor_alleles)

The peptide is predicted to be a non-binder for all of these alleles, except for DRB5\*01:01, for which it is predicted to be a strong binder.  
Here is the list of alleles without the prefixes:

In [ ]:
alleles = sorted(list(set([al[2:] for al in alleles_])))

We now filter the data to keep only peptides that are predicted to be weak or strong binders for the allele A\*02:01:

In [ ]:
allele_filtered_df = dfs["aggregated"][dfs["aggregated"].donor_alleles.apply(lambda x: ("w/A*02:01" in x) or ("s/A*02:01" in x))]

Here is the set of tissues in the database:

In [ ]:
tissues = set(np.concatenate([tissue.split(",") for tissue in dfs["aggregated"].tissues]))
tissues

We select two of them, for example "Thymus" and "Liver", and filter the data to keep only the peptides that have been found in either of these tissues:

In [ ]:
tissue_1 = "Lung"
tissue_2 = "Liver"
tissue_filtered_df = allele_filtered_df[allele_filtered_df.tissues.apply(lambda x: tissue_1 in x or tissue_2 in x)]
print(f"{len(tissue_filtered_df)} peptides")

In [ ]:
max_length_peptide = tissue_filtered_df.peptide_sequence.apply(len).max()
xd = sc.AnnData(shape=(tissue_filtered_df.shape[0], max_length_peptide * len(xomx.tl.aminoacids)))
xd.obs_names = np.array(tissue_filtered_df.peptide_sequence)
xd.X = np.empty((xd.n_obs, xd.n_vars))
for i in range(xd.n_obs):
    xd.X[i, :] = xomx.tl.onehot(xd.obs_names[i], max_length_peptide)
xd.obs['labels'] = np.array(tissue_filtered_df.tissues.apply(lambda x: (tissue_1 if tissue_1 in x else "") + (tissue_2 if tissue_2 in x else "")))
xd.uns['all_labels'] = xomx.tl.all_labels(xd.obs['labels'])
xd.uns['obs_indices_per_label'] = xomx.tl.indices_per_label(xd.obs['labels'])

In [ ]:
rng = np.random.RandomState(0)
xomx.pl.plot_2d_embedding(xd, trimap.TRIMAP())

In [ ]:
trimap.TRIMAP().transform()

In [ ]:
xomx.tl.train_and_test_indices(xd, "obs_indices_per_label", test_train_ratio=0.25, rng=rng)
classifier = {}
classifier[tissue_1] = xomx.fs.RFEExtraTrees(
    xd,
    tissue_1,
    n_estimators=450,
    random_state=rng,
)

In [ ]:
classifier[tissue_1].init()

In [ ]:
classifier[tissue_1].plot()

In [ ]:
xomx.tl.matthews_coef(classifier[tissue_1].confusion_matrix)

Remark: the MCC score obtained is close to 0.5, which is definitely better than random predictions (MCC ~ 0), however for other choices of alleles and tissues, we frequently obtain an MCC score close to 0, showing that the classifier is not able to generalize at all.  
The problem of tissue prediction based on HLA-presented peptides is hard, but there may be specific cases for which it is possible.

In [ ]:
predictor = mhcflurry.Class1PresentationPredictor.load()

In [ ]:
results1 = predictor.predict(["NLVPMVATV", "RANDMPEPTIDE"], ["A*02:01", "A*01:01", "A*03:01"])

In [ ]:
results1

In [ ]:
dfs["sample_hits"]

In [ ]:
donor_sample_hits = dfs["sample_hits"][dfs["sample_hits"].donor == "AUT01-DN02"]

In [ ]:
dfs["donors"][dfs["donors"].donor == "AUT01-DN02"]

In [ ]:
hla_class_filtered_df = donor_sample_hits[donor_sample_hits.hla_class == "HLA-I"]

In [ ]:
donor_filtered_df = dfs["aggregated"].take(np.array(sorted(list(set(hla_class_filtered_df.peptide_sequence_id)))) - 1)

In [ ]:
donor_filtered_df

In [ ]:
donor_filtered_max_length_peptide = donor_filtered_df.peptide_sequence.apply(len).max()
donor_filtered_xd = sc.AnnData(shape=(donor_filtered_df.shape[0], donor_filtered_max_length_peptide * len(xomx.tl.aminoacids)))
donor_filtered_xd.obs_names = np.array(donor_filtered_df.peptide_sequence)
donor_filtered_xd.X = np.empty((donor_filtered_xd.n_obs, donor_filtered_xd.n_vars))
for i in range(xd.n_obs):
    donor_filtered_xd.X[i, :] = xomx.tl.onehot(donor_filtered_xd.obs_names[i], donor_filtered_max_length_peptide)
# donor_filtered_xd.obs['labels'] = np.array(tissue_filtered_df.tissues.apply(lambda x: (tissue_1 if tissue_1 in x else "") + (tissue_2 if tissue_2 in x else "")))
# donor_filtered_xd.uns['all_labels'] = xomx.tl.all_labels(donor_filtered_xd.obs['labels'])
# donor_filtered_xd.uns['obs_indices_per_label'] = xomx.tl.indices_per_label(donor_filtered_xd.obs['labels'])

In [ ]:
len(list(donor_filtered_xd.obs_names))

In [ ]:
results1 = predictor.predict(list(donor_filtered_xd.obs_names), [alleles[21]])

In [ ]:
results1[results1.presentation_score > 0.9]

In [ ]:
results = {}
for al in alleles[:51]:
    results[al] = predictor.predict(list(donor_filtered_xd.obs_names), [al])

In [ ]:
for i in range(51):
    print(f"{i} ({alleles[i]}): {len(results[alleles[i]][results[alleles[i]].presentation_score > 0.98])}")

In [ ]:
predictor.predict(list(donor_filtered_xd.obs_names), [alleles[0]])

In [ ]:
xomx.pl.plot_2d_embedding(donor_filtered_xd, trimap.TRIMAP(distance="manhattan"))